In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
peopleDF = spark.read.parquet("/FileStore/tables/test/people-10m.parquet")
peopleWithFixedSalariesDF = peopleDF.select("firstName","middleName","lastName","gender","birthDate","ssn", abs(col("salary")).alias("salary"))

#칼럼추가 :  withColumn
peopleWithFixedSalaries20DF = peopleWithFixedSalariesDF.select("*").withColumn("salary10K",round(col("salary") /10000)).\
                                                        filter("salary >= 20000")

In [0]:
peopleWithFixedSalaries20DF.show(5)

+---------+----------+----------+------+-------------------+-----------+------+---------+
firstName|middleName| lastName|gender| birthDate| ssn|salary|salary10K|
+---------+----------+----------+------+-------------------+-----------+------+---------+
 Pennie| Carry|Hirschmann| F|1955-07-02 04:00:00|981-43-9345| 56172| 6.0|
 An| Amira| Cowper| F|1992-02-08 05:00:00|978-97-8086| 40203| 4.0|
 Quyen| Marlen| Dome| F|1970-10-11 04:00:00|957-57-8246| 53417| 5.0|
 Coralie| Antonina| Marshal| F|1990-04-11 04:00:00|963-39-4885| 94727| 9.0|
 Terrie| Wava| Bonar| F|1980-01-16 05:00:00|964-49-8051| 79908| 8.0|
+---------+----------+----------+------+-------------------+-----------+------+---------+
only showing top 5 rows

In [0]:
l = [('Ankit',25), ('Jalfaizy',22), ('saurabh',20), ('Bala',26)]
rdd = sc.parallelize(l)
people = rdd.map(lambda x : Row(name=x[0], age = int(x[1])))
#rdd에서 dataframe만들기
schemaPeople = sqlContext.createDataFrame(people)

In [0]:
people.take(3)

Out[4]: [Row(name='Ankit', age=25),
 Row(name='Jalfaizy', age=22),
 Row(name='saurabh', age=20)]

In [0]:
schemaPeople.show(5)

+--------+---+
 name|age|
+--------+---+
 Ankit| 25|
Jalfaizy| 22|
 saurabh| 20|
 Bala| 26|
+--------+---+

In [0]:
geoip_rdd = sc.textFile("/FileStore/tables/test/geoip.txt")
geoip_rdd1 = (geoip_rdd\
              .map(lambda x : x.split(",")))

In [0]:
geoip_rdd2 = geoip_rdd1.map(lambda x : Row(ip = x[0],code = x[1],country = x[2]))
geoip_rdd2.take(3)

Out[7]: [Row(ip='194.120.126.123', code='NL', country='Netherlands'),
 Row(ip='94.126.119.173', code='FR', country='France'),
 Row(ip='193.46.123.412', code='RU', country='Russian Federation')]

In [0]:
geoipDF = sqlContext.createDataFrame(geoip_rdd2)
geoipDF.show()

+---------------+----+------------------+
 ip|code| country|
+---------------+----+------------------+
194.120.126.123| NL| Netherlands|
 94.126.119.173| FR| France|
 193.46.123.412| RU|Russian Federation|
 46.235.67.202| RU|Russian Federation|
 193.161.193.64| RU|Russian Federation|
+---------------+----+------------------+

In [0]:
geoipDF.select("country", "ip").where("country = 'Russian Federation'").show()

+------------------+--------------+
 country| ip|
+------------------+--------------+
Russian Federation|193.46.123.412|
Russian Federation| 46.235.67.202|
Russian Federation|193.161.193.64|
+------------------+--------------+

In [0]:
%%time
step1 = geoipDF.select("country","ip")

CPU times: user 1.32 ms, sys: 223 µs, total: 1.54 ms
Wall time: 14.1 ms

In [0]:
%%time
step2 = step1.where("country = 'Russian Federation'")

CPU times: user 169 µs, sys: 28 µs, total: 197 µs
Wall time: 3.49 ms

In [0]:
%%time
step3 = step2.show()

+------------------+--------------+
 country| ip|
+------------------+--------------+
Russian Federation|193.46.123.412|
Russian Federation| 46.235.67.202|
Russian Federation|193.161.193.64|
+------------------+--------------+

CPU times: user 602 µs, sys: 105 µs, total: 707 µs
Wall time: 342 ms

In [0]:
geoipDF.rdd.collect()

Out[13]: [Row(ip='194.120.126.123', code='NL', country='Netherlands'),
 Row(ip='94.126.119.173', code='FR', country='France'),
 Row(ip='193.46.123.412', code='RU', country='Russian Federation'),
 Row(ip='46.235.67.202', code='RU', country='Russian Federation'),
 Row(ip='193.161.193.64', code='RU', country='Russian Federation')]

In [0]:
geoipDF.rdd.\
     filter(lambda x : x.country == 'Russian Federation').\
     take(3)

Out[14]: [Row(ip='193.46.123.412', code='RU', country='Russian Federation'),
 Row(ip='46.235.67.202', code='RU', country='Russian Federation'),
 Row(ip='193.161.193.64', code='RU', country='Russian Federation')]

In [0]:
#결과는 같으나 where은 lazy evaluation
geoipDF.select("ip","country")\
        .where("country == 'Russian Federation'")\
        .show(5)

+--------------+------------------+
 ip| country|
+--------------+------------------+
193.46.123.412|Russian Federation|
 46.235.67.202|Russian Federation|
193.161.193.64|Russian Federation|
+--------------+------------------+

In [0]:
%%time
geoipDF.select("ip","country")\
        .filter("country == 'Russian Federation'")\
        .show(5)

+--------------+------------------+
 ip| country|
+--------------+------------------+
193.46.123.412|Russian Federation|
 46.235.67.202|Russian Federation|
193.161.193.64|Russian Federation|
+--------------+------------------+

CPU times: user 3.1 ms, sys: 0 ns, total: 3.1 ms
Wall time: 417 ms

In [0]:
#agg사용하기 -> count안에 인자 들어갈 때 
peopleDF.groupBy('firstName').agg(count('lastName').alias('lastName count')).show(3)

peopleDF.groupBy('firstName').agg({'lastName':'count'}).withColumnRenamed('count(lastName)','lastName count').show(3)

+---------+--------------+
firstName|lastName count|
+---------+--------------+
 Alayna| 1313|
 Melaine| 1237|
 Faye| 1350|
+---------+--------------+
only showing top 3 rows

+---------+--------------+
firstName|lastName count|
+---------+--------------+
 Alayna| 1313|
 Melaine| 1237|
 Faye| 1350|
+---------+--------------+
only showing top 3 rows

In [0]:
#length 함수 사용
peopleDF.groupBy(length('firstName')).agg(count('*')).show(3)

peopleDF.groupBy(length('firstName')).count().show(3)

+-----------------+--------+
length(firstName)|count(1)|
+-----------------+--------+
 6| 2633717|
 3| 428721|
 5| 2278227|
+-----------------+--------+
only showing top 3 rows

+-----------------+-------+
length(firstName)| count|
+-----------------+-------+
 6|2633717|
 3| 428721|
 5|2278227|
+-----------------+-------+
only showing top 3 rows

In [0]:
peopleDF.groupBy('firstName',length('firstName')).agg(count("*")).show(3)

+---------+-----------------+--------+
firstName|length(firstName)|count(1)|
+---------+-----------------+--------+
 Yolonda| 7| 1283|
 Joyce| 5| 1232|
 Kecia| 5| 1300|
+---------+-----------------+--------+
only showing top 3 rows

In [0]:
peopleDF.groupBy('firstName',length('firstName')).agg(count("*").alias("count")).show(3)

+---------+-----------------+-----+
firstName|length(firstName)|count|
+---------+-----------------+-----+
 Yolonda| 7| 1283|
 Joyce| 5| 1232|
 Kecia| 5| 1300|
+---------+-----------------+-----+
only showing top 3 rows

In [0]:
peopleDF.groupBy('firstName',length('firstName')).count().alias("count").show(3)

+---------+-----------------+-----+
firstName|length(firstName)|count|
+---------+-----------------+-----+
 Yolonda| 7| 1283|
 Joyce| 5| 1232|
 Kecia| 5| 1300|
+---------+-----------------+-----+
only showing top 3 rows

In [0]:
peopleDF.agg(min('salary'),max('salary'),avg('salary'),sum('salary')).show()

+-----------+-----------+-------------+------------+
min(salary)|max(salary)| avg(salary)| sum(salary)|
+-----------+-----------+-------------+------------+
 -26884| 180841|72633.0076033|726330076033|
+-----------+-----------+-------------+------------+

In [0]:
peopleDF.orderBy('firstName').groupBy('firstName','gender').agg(min('salary')).show(3)

+---------+------+-----------+
firstName|gender|min(salary)|
+---------+------+-----------+
Magdalena| F| 11963|
 Meda| F| 1845|
 Vena| F| 13333|
+---------+------+-----------+
only showing top 3 rows

In [0]:
peopleDF.select('firstName','gender','salary').orderBy('firstName').groupBy('firstName','gender').agg(count('*'),min('salary')).show(3)

+---------+------+--------+-----------+
firstName|gender|count(1)|min(salary)|
+---------+------+--------+-----------+
Magdalena| F| 1215| 11963|
 Meda| F| 1309| 1845|
 Vena| F| 1223| 13333|
+---------+------+--------+-----------+
only showing top 3 rows

In [0]:
#내림차순
peopleDF.orderBy('firstName').groupBy('firstName').agg(count('*').alias('count')).orderBy(col('count').desc()).show(3)

+---------+-----+
firstName|count|
+---------+-----+
 Sammie| 5462|
 Daryl| 5442|
 Bobbie| 5432|
+---------+-----+
only showing top 3 rows

In [0]:
#오름차순
peopleDF.orderBy('firstName').groupBy('firstName').agg(count('*').alias('count')).orderBy(col('count')).show(3)

+---------+-----+
firstName|count|
+---------+-----+
 Marylynn| 1150|
 Verna| 1154|
 Carlena| 1159|
+---------+-----+
only showing top 3 rows

In [0]:
#각종 파일 읽기
#cvs파일 읽기

#스키마로 받아오고 헤더 추가
df = (spark.read\
  .option("header", "true") \
  .option("inferSchema","true")\
  .csv("/FileStore/tables/test/day.csv"))

In [0]:
#json파일 읽기
#옵션 외우기
databricksBlogDF = spark\
                  .read.option("inferSchema","true")\
                  .option("header","true")\
                  .json("/FileStore/tables/databricks_blog-2.json")


In [0]:
datesDF = databricksBlogDF.select("dates")
display(datesDF)

dates
"List(2014-04-10, 2014-04-10, UTC)"
"List(2014-04-10, 2014-04-10, UTC)"
"List(2014-04-01, 2014-04-01, UTC)"
"List(2014-03-27, 2014-03-27, UTC)"
"List(2014-02-04, 2014-02-04, UTC)"
"List(2014-01-02, 2014-01-02, UTC)"
"List(2014-03-26, 2014-03-26, UTC)"
"List(2014-03-21, 2014-03-21, UTC)"
"List(2014-03-19, 2014-03-19, UTC)"
"List(2014-03-03, 2014-03-03, UTC)"


In [0]:
display(databricksBlogDF.select("authors","dates.publishedOn").\
       filter(year(col("dates.createdOn")) > "2014"))
                      

authors,publishedOn
List(by Databricks Press Office),2015-01-13
List(Kavitha Mariappan),2015-01-16
List(Kavitha Mariappan),2015-01-20
List(Yin Huai (Databricks)),2015-02-02
List(Jeremy Freeman (Howard Hughes Medical Institute)),2015-01-28
List(Dave Wang (Databricks)),2015-02-05
"List(Xiangrui Meng, Joseph Bradley, Evan Sparks (UC Berkeley), Shivaram Venkataraman (UC Berkeley))",2015-01-07
List(Michael Armbrust),2015-01-09
"List(Joseph K. Bradley (Databricks), Manish Amde (Origami Logic))",2015-01-21
List(Tathagata Das),2015-01-15


In [0]:
databricksBlog3DF = databricksBlogDF.withColumn("publishOn",col("dates.publishedOn"))

In [0]:
databricksBlog3DF.printSchema()

root
-- authors: array (nullable = true)
 |-- element: string (containsNull = true)
-- categories: array (nullable = true)
 |-- element: string (containsNull = true)
-- content: string (nullable = true)
-- creator: string (nullable = true)
-- dates: struct (nullable = true)
 |-- createdOn: string (nullable = true)
 |-- publishedOn: string (nullable = true)
 |-- tz: string (nullable = true)
-- description: string (nullable = true)
-- id: long (nullable = true)
-- link: string (nullable = true)
-- slug: string (nullable = true)
-- status: string (nullable = true)
-- title: string (nullable = true)
-- publishOn: string (nullable = true)

In [0]:
#Casting
databricksBlogDF.select("title",date_format("dates.publishedOn","yyyy-MM-dd")\
                       .alias("publishedOn")).show(5)

+--------------------+-----------+
 title|publishedOn|
+--------------------+-----------+
MapR Integrates t...| 2014-04-10|
Apache Spark 0.9....| 2014-04-10|
Application Spotl...| 2014-04-01|
Spark SQL: Manipu...| 2014-03-27|
Apache Spark 0.9....| 2014-02-04|
+--------------------+-----------+
only showing top 5 rows

In [0]:
databricksBlogDF.select("title",col("dates.publishedOn")).show(5)

+--------------------+-----------+
 title|publishedOn|
+--------------------+-----------+
MapR Integrates t...| 2014-04-10|
Apache Spark 0.9....| 2014-04-10|
Application Spotl...| 2014-04-01|
Spark SQL: Manipu...| 2014-03-27|
Apache Spark 0.9....| 2014-02-04|
+--------------------+-----------+
only showing top 5 rows

In [0]:
databricksBlog2DF = databricksBlogDF.withColumn("publishedOn",date_format("dates.publishedOn","yyyy-MM-dd"))

In [0]:
resultDF = (databricksBlog2DF.select("title",to_date(col("publishedOn")).alias('date'),"link").\
                            filter(year(col("publishedOn"))=='2013').\
                            orderBy(col("publishedOn")))
display(resultDF)
                  

title,date,link
Databricks and the Apache Spark Platform,2013-10-27,https://databricks.com/blog/2013/10/27/databricks-and-the-apache-spark-platform.html
The Growing Apache Spark Community,2013-10-28,https://databricks.com/blog/2013/10/27/the-growing-spark-community.html
Databricks and Cloudera Partner to Support Apache Spark,2013-10-29,https://databricks.com/blog/2013/10/28/databricks-and-cloudera-partner-to-support-spark.html
Putting Apache Spark to Use: Fast In-Memory Computing for Your Big Data Applications,2013-11-22,https://databricks.com/blog/2013/11/21/putting-spark-to-use.html
Highlights From Spark Summit 2013,2013-12-19,https://databricks.com/blog/2013/12/18/spark-summit-2013-follow-up.html
Apache Spark 0.8.1 Released,2013-12-20,https://databricks.com/blog/2013/12/19/release-0_8_1.html


In [0]:
resultDF = (databricksBlog2DF.select("title",col("publishedOn").alias('date'),"link").\
                            filter(year(col("publishedOn"))=='2013').\
                            orderBy(col("publishedOn")))
display(resultDF)

title,date,link
Databricks and the Apache Spark Platform,2013-10-27,https://databricks.com/blog/2013/10/27/databricks-and-the-apache-spark-platform.html
The Growing Apache Spark Community,2013-10-28,https://databricks.com/blog/2013/10/27/the-growing-spark-community.html
Databricks and Cloudera Partner to Support Apache Spark,2013-10-29,https://databricks.com/blog/2013/10/28/databricks-and-cloudera-partner-to-support-spark.html
Putting Apache Spark to Use: Fast In-Memory Computing for Your Big Data Applications,2013-11-22,https://databricks.com/blog/2013/11/21/putting-spark-to-use.html
Highlights From Spark Summit 2013,2013-12-19,https://databricks.com/blog/2013/12/18/spark-summit-2013-follow-up.html
Apache Spark 0.8.1 Released,2013-12-20,https://databricks.com/blog/2013/12/19/release-0_8_1.html


In [0]:
#column에 array
display(databricksBlogDF.select(size("authors"),"authors"))

size(authors),authors
1,List(Tomer Shiran (VP of Product Management at MapR))
1,List(Tathagata Das)
1,List(Steven Hillion)
2,"List(Michael Armbrust, Reynold Xin)"
1,List(Patrick Wendell)
2,"List(Ali Ghodsi, Ahir Reddy)"
2,"List(Russell Cardullo (Data Infrastructure Engineer at Sharethrough), Michael Ruggiero (Data Infrastructure Engineer at Sharethrough))"
2,"List(Jai Ranganathan, Matei Zaharia)"
1,List(Databricks Press Office)
1,List(Ion Stoica)


In [0]:
display(databricksBlogDF.select(col("authors")[0].alias("primaryAuthor")))#****************************원소접근

primaryAuthor
Tomer Shiran (VP of Product Management at MapR)
Tathagata Das
Steven Hillion
Michael Armbrust
Patrick Wendell
Ali Ghodsi
Russell Cardullo (Data Infrastructure Engineer at Sharethrough)
Jai Ranganathan
Databricks Press Office
Ion Stoica


In [0]:
#여러개의 row가 생성되어 보여짐
display(databricksBlogDF.select("title","authors",explode(col("authors")).alias("author"),"link"))

title,authors,author,link
MapR Integrates the Complete Apache Spark Stack,List(Tomer Shiran (VP of Product Management at MapR)),Tomer Shiran (VP of Product Management at MapR),https://databricks.com/blog/2014/04/10/mapr-integrates-spark-stack.html
Apache Spark 0.9.1 Released,List(Tathagata Das),Tathagata Das,https://databricks.com/blog/2014/04/09/spark-0_9_1-released.html
Application Spotlight: Alpine Data Labs,List(Steven Hillion),Steven Hillion,https://databricks.com/blog/2014/03/31/application-spotlight-alpine.html
Spark SQL: Manipulating Structured Data Using Apache Spark,"List(Michael Armbrust, Reynold Xin)",Michael Armbrust,https://databricks.com/blog/2014/03/26/spark-sql-manipulating-structured-data-using-spark-2.html
Spark SQL: Manipulating Structured Data Using Apache Spark,"List(Michael Armbrust, Reynold Xin)",Reynold Xin,https://databricks.com/blog/2014/03/26/spark-sql-manipulating-structured-data-using-spark-2.html
Apache Spark 0.9.0 Released,List(Patrick Wendell),Patrick Wendell,https://databricks.com/blog/2014/02/03/release-0_9_0.html
Apache Spark In MapReduce (SIMR),"List(Ali Ghodsi, Ahir Reddy)",Ali Ghodsi,https://databricks.com/blog/2014/01/01/simr.html
Apache Spark In MapReduce (SIMR),"List(Ali Ghodsi, Ahir Reddy)",Ahir Reddy,https://databricks.com/blog/2014/01/01/simr.html
Sharethrough Uses Apache Spark Streaming to Optimize Bidding in Real Time,"List(Russell Cardullo (Data Infrastructure Engineer at Sharethrough), Michael Ruggiero (Data Infrastructure Engineer at Sharethrough))",Russell Cardullo (Data Infrastructure Engineer at Sharethrough),https://databricks.com/blog/2014/03/25/sharethrough-and-spark-streaming.html
Sharethrough Uses Apache Spark Streaming to Optimize Bidding in Real Time,"List(Russell Cardullo (Data Infrastructure Engineer at Sharethrough), Michael Ruggiero (Data Infrastructure Engineer at Sharethrough))",Michael Ruggiero (Data Infrastructure Engineer at Sharethrough),https://databricks.com/blog/2014/03/25/sharethrough-and-spark-streaming.html


In [0]:
display(databricksBlogDF.select("title","authors",explode(col("authors")).alias("author"),"link")
       .filter(size(col("authors")) > 1)
       .orderBy("title")
       )

title,authors,author,link
"""Learning Spark"" book available from O'Reilly","List(Holden Karau, Andy Konwinski, Patrick Wendell, Matei Zaharia)",Matei Zaharia,https://databricks.com/blog/2015/02/09/learning-spark-book-available-from-oreilly.html
"""Learning Spark"" book available from O'Reilly","List(Holden Karau, Andy Konwinski, Patrick Wendell, Matei Zaharia)",Holden Karau,https://databricks.com/blog/2015/02/09/learning-spark-book-available-from-oreilly.html
"""Learning Spark"" book available from O'Reilly","List(Holden Karau, Andy Konwinski, Patrick Wendell, Matei Zaharia)",Andy Konwinski,https://databricks.com/blog/2015/02/09/learning-spark-book-available-from-oreilly.html
"""Learning Spark"" book available from O'Reilly","List(Holden Karau, Andy Konwinski, Patrick Wendell, Matei Zaharia)",Patrick Wendell,https://databricks.com/blog/2015/02/09/learning-spark-book-available-from-oreilly.html
AMPLab updates the Big Data Benchmark,"List(Ahir Reddy, Reynold Xin)",Ahir Reddy,https://databricks.com/blog/2014/02/12/big-data-benchmark.html
AMPLab updates the Big Data Benchmark,"List(Ahir Reddy, Reynold Xin)",Reynold Xin,https://databricks.com/blog/2014/02/12/big-data-benchmark.html
Announcing Apache Spark Packages,"List(Xiangrui Meng, Patrick Wendell)",Patrick Wendell,https://databricks.com/blog/2014/12/22/announcing-spark-packages.html
Announcing Apache Spark Packages,"List(Xiangrui Meng, Patrick Wendell)",Xiangrui Meng,https://databricks.com/blog/2014/12/22/announcing-spark-packages.html
Apache Spark 1.1: Bringing Hadoop Input/Output Formats to PySpark,"List(Nick Pentreath (Graphflow), Kan Zhang (IBM))",Nick Pentreath (Graphflow),https://databricks.com/blog/2014/09/17/spark-1-1-bringing-hadoop-inputoutput-formats-to-pyspark.html
Apache Spark 1.1: Bringing Hadoop Input/Output Formats to PySpark,"List(Nick Pentreath (Graphflow), Kan Zhang (IBM))",Kan Zhang (IBM),https://databricks.com/blog/2014/09/17/spark-1-1-bringing-hadoop-inputoutput-formats-to-pyspark.html
